# Notebook Application for Weekly Research on Daily Portfolio Companies using Tavily and LangGraph Agents

## Libraries

In [1]:
import os
import json
import asyncio
from typing import List, Optional 
from datetime import datetime


from tavily import AsyncTavilyClient
from IPython.display import Markdown, display

from langgraph.types import Command
from langchain_core.runnables import RunnableConfig
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_core.messages import AIMessage
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, END, MessagesState

## Set API KEYS

In [2]:
#should include 2 lines; TAVILY_API_KEY= and OPENAI_API_KEY=
from dotenv import load_dotenv
load_dotenv('.env')

True

### Tavily client & results

In [3]:
tavily_client = AsyncTavilyClient()
query_with_date = f"{"What are the school districts in seattle metro?"} {datetime.now().strftime('%m-%Y')}"
response = await tavily_client.search(query=query_with_date, max_results=5)

In [4]:
response

{'query': 'What are the school districts in seattle metro? 07-2025',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.niche.com/k12/search/best-school-districts/m/seattle-metro-area/',
   'title': '2025 Best School Districts in the Seattle Area - Niche',
   'content': 'Read 160 reviews * grade A+ Overall Niche Grade * Total schools 32 * Students 19,089 Bellevue School District ------------------------ #1 Best School Districts in Seattle Area. Read 115 reviews * grade A+ Overall Niche Grade * Total schools 30 * Students 19,585 Issaquah School District ------------------------ #4 Best School Districts in Seattle Area. Read 34 reviews * grade A Overall Niche Grade * Total schools 8 * Students 5,619 University Place School District -------------------------------- #7 Best School Districts in Seattle Area. Read 49 reviews * grade B+ Overall Niche Grade * Total schools 14 * Students 5,418 North Kitsap School District ----------------------------

## Setup the Graph


#used in Plan which is used in State
class Step(BaseModel):
    title: str
    description: str = Field(..., description="Specify exactly what data to collect")
    execution_res: Optional[str] = Field(
        default=None, description="The Step execution result"
    )

#used in the State object
class Plan(BaseModel):
    thought: str
    title: str
    steps: List[Step] = Field(
        default_factory=list,
        description="Research & Processing steps to get more context",
    )

class Resource(BaseModel):
    """
    Resource is a class that represents a resource.
    """

    uri: str = Field(..., description="The URI of the resource")
    title: str = Field(..., description="The title of the resource")
    description: str | None = Field("", description="The description of the resource")

class State(MessagesState):
    """State for our research agents."""
    research_topic: str = ""
    observations: list[str] = []
    queries: list[str] = []
    resources: list[Resource] = []
    plan_iterations: int = 0
    current_plan: Plan | str = None
    final_report: str = ""

### LLM Configuration

In [5]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=4,  
    check_every_n_seconds=0.1,
    max_bucket_size=10,
)

basic_llm = init_chat_model("openai:gpt-4.1-mini", rate_limiter=rate_limiter)
reasoning_llm = init_chat_model("openai:gpt-4.1-mini", rate_limiter=rate_limiter)

## Graph Setup

In [6]:
#used in Plan which is used in State
class Step(BaseModel):
    title: str
    description: str = Field(..., description="Specify exactly what data to collect")
    execution_res: Optional[str] = Field(
        default=None, description="The Step execution result"
    )

#used in the State object
class Plan(BaseModel):
    thought: str
    title: str
    steps: List[Step] = Field(
        default_factory=list,
        description="Research & Processing steps to get more context",
    )

class Resource(BaseModel):
    """
    Resource is a class that represents a resource.
    """

    uri: str = Field(..., description="The URI of the resource")
    title: str = Field(..., description="The title of the resource")
    description: str | None = Field("", description="The description of the resource")

class State(MessagesState):
    """State for our research agents."""
    research_topic: str = ""
    observations: list[str] = []
    queries: list[str] = []
    resources: list[Resource] = []
    plan_iterations: int = 0
    current_plan: Plan | str = None
    final_report: str = ""

## Prompt Management

In [7]:
with open("planner.md", "r", encoding="utf-8") as file:
    plan_prompt = file.read()

In [8]:
from jinja2 import Template

template = Template(plan_prompt)

rendered = template.render(date=datetime.now().strftime('%m/%d/%Y'), max_step_num=10)

print(rendered)
# Output: Maximum steps allowed: 10

planning_prompt = """Today's date is 07/13/2025.  You are a professional research agent.  Plan information gathering and research tasks using a team of specialized agents to collect
relevant data for preparing an in depth report on a given topic or request.
Break down the request into specific tasks or activities that can be pursued by researchers to gather information for subsequent analysis, it is critical
to be thorough in information collection, sampling a broad range of information or topics as well as in depth detail or analysis for given
portions of the topic or question.

## Information Standards and Success
A successful research plan must meet these requirements:
1. Comprehensive Coverage
-Explore all aspects of the topic to identify associated themes or trends that shed light on the question at hand
-Represent multiple viewpoints or perspectives

2. Detailed Analysis
-For key portions of the question or topic, perform detailed analysis, performing research to further explore 

In [9]:
question = """What are the rankings for the broader Seattle metro region's schools?  
Which stand out, which are lagging and what are some common themes or drivers for those that lead or lag?  
How should families evaluate their options including private schools in some of the key areas of the metro?"""

messages = [{"role": "system", "content": rendered}, {"role": "user", "content": question}]

In [10]:
full_response = ""
plan_llm = reasoning_llm.with_structured_output(Plan,method="json_mode")
response = plan_llm.invoke(messages)
full_response = response.model_dump_json(indent=4, exclude_none=True)

json.loads(full_response)

{'thought': 'The user is requesting a detailed research plan to gather comprehensive data on school rankings across the broader Seattle metro area, identifying both top-performing and lagging schools, understanding the factors influencing these standings, and advising on how families can evaluate school options including private schools in key metro areas.',
 'title': 'Research Plan for Seattle Metro Area School Rankings and Family Evaluation Strategies',
 'steps': [{'title': 'Collect Comprehensive School Rankings and Performance Data',
   'description': 'Gather detailed rankings and performance metrics for all public and private schools in the broader Seattle metro area, including standardized test scores, graduation rates, college admission rates, and school rating systems from sources like GreatSchools, Niche, and state education departments. Include diverse viewpoints such as academic, extracurricular, and special program ratings.'},
  {'title': 'Identify Top-Performing and Lagging

## Planning Graph

In [11]:
from dataclasses import dataclass, field, fields

#used in apply_prompt_templates
@dataclass(kw_only=True)
class Configuration:
    """The configurable fields."""
    max_plan_iterations: int = 1  # Maximum number of plan iterations
    max_step_num: int = 3  # Maximum number of steps in a plan
    max_search_results: int = 3  # Maximum number of search results
    mcp_settings: dict = None  # MCP settings, including dynamic loaded tools

    @classmethod
    def from_runnable_config(
        cls, config: Optional[RunnableConfig] = None
    ) -> "Configuration":
        """Create a Configuration instance from a RunnableConfig."""
        configurable = (
            config["configurable"] if config and "configurable" in config else {}
        )
        values: dict[str, Any] = {
            f.name: os.environ.get(f.name.upper(), configurable.get(f.name))
            for f in fields(cls)
            if f.init
        }
        return cls(**{k: v for k, v in values.items() if v})

In [12]:
initial_state = {
        #Setting up State
        "messages": [{"role": "user", "content": question}],
        "documents": []
    }

config = {
    #runtime configuration or variables
        "configurable": {
            "thread_id": "default",
            "max_plan_iterations": 1,
            "max_step_num": 5,
        },
        "recursion_limit": 100,
    }


In [13]:
def prepare_prompt(node, config, messages):
    with open(f"{node}.md", "r", encoding="utf-8") as file:
        plan_prompt = file.read()
    template = Template(plan_prompt)

    prompt = template.render(date=datetime.now().strftime('%d/%m/%Y'), max_step_num=config.max_step_num)
    return [{"role": "system", "content": prompt}] + messages

In [14]:
def planner_node(state: State, config: RunnableConfig):
    configurable = Configuration.from_runnable_config(config)
    messages = state.get("messages")
    plan_llm = reasoning_llm.with_structured_output(Plan,method="json_mode")
    #if plan_iterations >= configurable.max_plan_iterations:
    #    return Command(goto="reporter")
    messages = prepare_prompt("planner",configurable,messages)
    full_response = ""
    response = plan_llm.invoke(messages)
    full_response = response.model_dump_json(indent=4, exclude_none=True)
    plan_response = json.loads(full_response)
    new_plan = Plan.model_validate(plan_response)
    return Command(
        update={
            "messages": [AIMessage(content=full_response, name="planner")],
            "current_plan": new_plan,
        },
        goto="research_coordinator",
    )

In [15]:
def research_coordinator_node(state: State):
    print("Nothing")
    """Research team node that collaborates on tasks."""
    pass

In [16]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

graph = StateGraph(State)

# Add nodes
graph.add_node("planner", planner_node)
graph.add_node("research_coordinator", research_coordinator_node)
graph.set_entry_point("planner")
graph.add_edge("research_coordinator", END)  # Option in the future, to add another step and filter the documents retrieved using rerhank before writing the report

agent = graph.compile(checkpointer=memory)

In [17]:
events = list(agent.stream(input=initial_state, config=config, stream_mode="values"))

Nothing


In [18]:
snapshot = agent.get_state(config)
{k for k, v in snapshot.values.items()}

{'current_plan', 'messages'}

In [19]:
snapshot.values['current_plan'].steps

[Step(title='Collect Comprehensive Data on Seattle Metro School Rankings', description='Gather detailed rankings and performance data for all types of schools in the Seattle metro region, including public, charter, and private institutions. Data should include academic achievement scores, graduation rates, standardized test scores, college readiness indicators, and extra-curricular offerings from sources such as state education departments, GreatSchools, Niche, and relevant school district reports.', execution_res=None),
 Step(title='Identify High-Performing and Lagging Schools with Analysis of Common Drivers', description='Analyze the data to identify schools that consistently rank at the top or bottom. Collect qualitative and quantitative information regarding factors influencing performance such as funding levels, teacher qualifications and retention, student demographics, community involvement, resource availability, and administrative policies. Include case studies and expert opin

## Researcher

### Research Prompt

In [20]:
def research_prompt(node, thought, step):
    with open(f"{node}.md", "r", encoding="utf-8") as file:
        plan_prompt = file.read()
    template = Template(plan_prompt)
    prompt = template.render(date=datetime.now().strftime('%d/%m/%Y'), thought=thought, step=step)
    return prompt

In [21]:
plan = snapshot.values['current_plan']
plan_steps = []
for i in plan.steps:
    plan_steps.append(research_prompt("researcher",plan.thought,i))

In [22]:
plan_steps[0]

'planning_prompt = """The current date is 13/07/2025.\n\nYou are a research agent that is managed by a supervisory agent.  You conduct thorough investigations using web search tools in order to collect information that will be synthesized into a comprehensive report on a given topic.\n\n# Instructions\n- Assess the \'Step\' below to evaluate if a web search is required in order to obtain more information or answer the question.  All questions or requests for more information require searches and must be cited, innate knowledge or assumptions cannot be used.\n- Use the information provided in order to construct one or more search queries that you expect will produce relevant results or more context or information about the topic or query.\n- Your queries should be succinct and to the point, do not use multiple sentences or overlapping and possibly conflicting instructions, the query should be as direct and short as possible in order to find relevant results.\n- Reply with only the query

### Research Tools

In [23]:
class Query(BaseModel):
    query: str = Field(description="web search query")

# Define the args_schema for the tavily_search tool using a multi-query approach, enabling more precise queries for Tavily.
class SearchInput(BaseModel):
    sub_queries: List[Query] = Field(description="set of sub-queries that can be answered in isolation")
class ExtractInput(BaseModel):
    urls: List[str] = Field(description="list of a single or several URLs for extracting raw content to gather additional information")


@tool("web_search", args_schema=SearchInput, return_direct=True)
async def web_search(sub_queries: List[Query]):
    """Perform searches for each sub-query using the Tavily search tool concurrently."""  
    # Define a coroutine function to perform a single search with error handling
    async def perform_search(itm):
        try:
            # Attempt to perform the search, hardcoding days to 7 (days will be used only when topic is news)
            print(f'searching for %s' % itm)
            response = await tavily_client.search(query=itm, max_results=10)
            return response['results']
        except Exception as e:
            # Handle any exceptions, log them, and return an empty list
            print(f"Error occurred during search for query '{itm}': {str(e)}")
            return []
    
    # Run all the search tasks in parallel
    search_tasks = [perform_search(itm.query) for itm in sub_queries]
    search_responses = await asyncio.gather(*search_tasks)
    
    # Combine the results from all the responses
    search_results = []
    for response in search_responses:
        search_results.extend(response)
    
    return search_results

tools = [web_search]
tools_by_name = {tool.name: tool for tool in tools}
tavily_client = AsyncTavilyClient()

In [24]:
tools_by_name

{'web_search': StructuredTool(name='web_search', description='Perform searches for each sub-query using the Tavily search tool concurrently.', args_schema=<class '__main__.SearchInput'>, return_direct=True, coroutine=<function web_search at 0x10b5be980>)}

### An example tool call

In [25]:
tool_call_example = {'name': 'web_search',
    'args': {'sub_queries': [{'query': 'criteria families should consider when choosing schools in Seattle metro area'},
      {'query': 'top private schools in Seattle metro area compared to public schools'},
      {'query': 'school choice and enrollment resources in Seattle metro area'},
      {'query': 'factors for evaluating school fit based on family values and student needs'},
      {'query': 'Seattle metro area school safety and transportation options'}]},
    'id': 'call_LwRdX7dWXO8qZeAQc0vBaDMf',
    'type': 'tool_call'}
new_docs = await tools_by_name['web_search'].ainvoke(tool_call_example["args"])

searching for criteria families should consider when choosing schools in Seattle metro area
searching for top private schools in Seattle metro area compared to public schools
searching for school choice and enrollment resources in Seattle metro area
searching for factors for evaluating school fit based on family values and student needs
searching for Seattle metro area school safety and transportation options


In [26]:
new_docs[:3]

[{'url': 'https://www.seattleschools.org/enroll/about-our-schools/school-choice/',
  'title': 'School Choice and Open Enrollment - Seattle Public Schools',
  'content': 'Families can also apply to request that their student attend a different school through the Open Enrollment for School Choice Process. School Choice Open Enrollment Video Families with students enrolled in Seattle Public Schools may submit school choice applications for the following school year. Will have a guaranteed HC Pathway seat at their HC Pathway High School (Garfield, Lincoln, or West Seattle),\u202fbut a Choice Form must be submitted during Open Enrollment. Students receive highest priority for assignment if they have a sibling attending for the current and following school year that choice school they are applying to. Students who do not receive an assignment are placed on a waitlist for their first-choice school only.',
  'score': 0.5996498,
  'raw_content': None},
 {'url': 'https://www.reddit.com/r/Seattle

In [27]:
#transforming these results into Resources, which we'll then store in the State
resource_list = []
for doc in new_docs:
    resource_list.append(Resource(uri=doc['url'], title = doc['title'], description=doc['content']))
resource_list[:3]

[Resource(uri='https://www.seattleschools.org/enroll/about-our-schools/school-choice/', title='School Choice and Open Enrollment - Seattle Public Schools', description='Families can also apply to request that their student attend a different school through the Open Enrollment for School Choice Process. School Choice Open Enrollment Video Families with students enrolled in Seattle Public Schools may submit school choice applications for the following school year. Will have a guaranteed HC Pathway seat at their HC Pathway High School (Garfield, Lincoln, or West Seattle),\u202fbut a Choice Form must be submitted during Open Enrollment. Students receive highest priority for assignment if they have a sibling attending for the current and following school year that choice school they are applying to. Students who do not receive an assignment are placed on a waitlist for their first-choice school only.'),
 Resource(uri='https://www.reddit.com/r/Seattle/comments/1is4pj5/greater_seattle_area_pa

## A Graph with Research

In [28]:
async def research_node(state: State, config: RunnableConfig):
    resources = state.get('resources',[])
    full_response = ""
    for tool_call in state["queries"]:
        tool_call = tool_call[0]
        tool = tools_by_name[tool_call["name"]]
        new_docs = await tool.ainvoke(tool_call["args"])
        for doc in new_docs:
            resource_list.append(Resource(uri=doc['url'], title = doc['title'], description=doc['content']))
    return {"resources": resource_list}

In [29]:
async def research_coordinator_node(state: State, config: RunnableConfig):
    messages = state.get("messages")
    print("running research")
    plan = state.get("current_plan")
    research_results = []
    for i in plan.steps:
        plan_prompt = research_prompt("researcher",plan.thought,i)
        response = basic_llm.bind_tools(tools).invoke(plan_prompt)
        research_results.append(response.tool_calls)
    return {"queries": research_results}

In [30]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

graph = StateGraph(State)

# Add nodes
graph.add_node("planner", planner_node)
graph.add_node("research_coordinator", research_coordinator_node)
graph.add_node("researcher",research_node)

graph.set_entry_point("planner")
graph.add_edge("research_coordinator", "researcher")
graph.add_edge("researcher", END)  # Option in the future, to add another step and filter the documents retrieved using rerhank before writing the report

agent = graph.compile(checkpointer=memory)

async for s in agent.astream(input=initial_state, config=config, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the rankings for the broader Seattle metro region's schools?  
Which stand out, which are lagging and what are some common themes or drivers for those that lead or lag?  
How should families evaluate their options including private schools in some of the key areas of the metro?
================================== Ai Message ==================================
Name: planner

{
    "thought": "The user's request is to gather comprehensive information on the rankings of schools in the broader Seattle metro region, identify standout and lagging schools, analyze common themes or drivers behind their performance, and provide guidance on how families should evaluate school options including private schools in key metro areas.",
    "title": "Research Plan for Seattle Metro Region School Rankings and Evaluation",
    "steps": [
        {
            "title": "Collect Current and Historical School Rankings 

In [31]:
#graph steps and message collection
for state in agent.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)

Num Messages:  2 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('researcher',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('research_coordinator',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('planner',)
--------------------------------------------------------------------------------
Num Messages:  0 Next:  ('__start__',)
--------------------------------------------------------------------------------


In [32]:
snapshot = agent.get_state(config)
{k for k, v in snapshot.values.items()}

{'current_plan', 'messages', 'queries', 'resources'}

In [33]:
snapshot.values['current_plan'].thought

"The user's request is to gather comprehensive information on the rankings of schools in the broader Seattle metro region, identify standout and lagging schools, analyze common themes or drivers behind their performance, and provide guidance on how families should evaluate school options including private schools in key metro areas."

In [34]:
snapshot.values['resources'][:5]

[Resource(uri='https://www.seattleschools.org/enroll/about-our-schools/school-choice/', title='School Choice and Open Enrollment - Seattle Public Schools', description='Families can also apply to request that their student attend a different school through the Open Enrollment for School Choice Process. School Choice Open Enrollment Video Families with students enrolled in Seattle Public Schools may submit school choice applications for the following school year. Will have a guaranteed HC Pathway seat at their HC Pathway High School (Garfield, Lincoln, or West Seattle),\u202fbut a Choice Form must be submitted during Open Enrollment. Students receive highest priority for assignment if they have a sibling attending for the current and following school year that choice school they are applying to. Students who do not receive an assignment are placed on a waitlist for their first-choice school only.'),
 Resource(uri='https://www.reddit.com/r/Seattle/comments/1is4pj5/greater_seattle_area_pa

## Collection & Downsizing

In [35]:

with open(f"collection.md", "r", encoding="utf-8") as file:
    plan_prompt = file.read()
template = Template(plan_prompt)
thought = snapshot.values["current_plan"].thought
step = snapshot.values["current_plan"].steps
query = snapshot.values["queries"]
resource = snapshot.values["resources"]
prompt = template.render(date=datetime.now().strftime('%d/%m/%Y'), thought=thought, step=step, query=query, resource=resource)

relevant_urls = reasoning_llm.with_structured_output(ExtractInput).invoke(prompt)

In [36]:
relevant_urls

ExtractInput(urls=['https://www.niche.com/k12/search/best-public-elementary-schools/m/seattle-metro-area/', 'https://www.niche.com/k12/search/best-private-high-schools/m/seattle-metro-area/', 'https://www.usnews.com/education/best-high-schools/washington/rankings/seattle-wa-42660', 'https://www.seattleschools.org/departments/assessments/', 'https://ospi.k12.wa.us/data-reporting', 'https://saveseattleschools.blogspot.com/2025/04/enrollment-trends-for-seattle-schools.html', 'https://www.nctq.org/wp-content/uploads/2025/03/Human_Capital_in_Seattle_Public_Schools_NCTQ_Report.pdf', 'https://www.seattleschools.org/news/class-size-adjustments/', 'https://www.seattleschools.org/wp-content/uploads/2025/02/Enrollment-Decline-Study_ADA.pdf', 'https://www.seattleschools.org/departments/community-partnerships/', 'https://www.seattletimes.com/education-lab/why-have-thousands-left-seattle-schools-a-new-study-suggests-answers/', 'https://www.collegetransitions.com/blog/best-seattle-private-schools/', 

In [37]:
def collection_prompt(node, state):
    with open(f"{node}.md", "r", encoding="utf-8") as file:
        plan_prompt = file.read()
    template = Template(plan_prompt)
    thought = state.get("current_plan").thought
    step = state.get("current_plan").steps
    query = state.get("queries")
    resource = state.get("resources")
    prompt = template.render(date=datetime.now().strftime('%d/%m/%Y'), thought=thought, step=step, query=query, resource=resource)
    return prompt

### Document Extraction

In [40]:
response = []
for url_items in relevant_urls.urls:
    response.append(await tavily_client.extract(urls=url_items))
    break

In [42]:
response

[{'results': [{'url': 'https://www.niche.com/k12/search/best-public-elementary-schools/m/seattle-metro-area/',
    'raw_content': '2025 Best Public Elementary Schools in the Seattle Area - Niche\n\n===============\n\n[Skip to Main Content](https://www.niche.com/k12/search/best-public-elementary-schools/m/seattle-metro-area/#maincontent)\n\n[](https://www.niche.com/?ref=k12 "Niche")\n\nK-12 Schools Seattle Area\n\nProvides auto-suggestions when entering text\n\nFind a school or district ...\n\nProvides auto-suggestions when entering text\n\nSearch in a district or a location ...\n\nLog In Sign Up\n\n*   [School Search](https://www.niche.com/k12/search/best-schools/m/seattle-metro-area/)\n*   [School Rankings](https://www.niche.com/k12/rankings/)\n*   [Schools Near You](https://www.niche.com/k12/schools-near-you/)\n\n[Write a review](https://www.niche.com/survey/hub/)\n\n[K-12 SCHOOLS](https://www.niche.com/k12/search/best-schools/m/seattle-metro-area/)\n\n*   [Review Your School](https:

In [43]:
async def collection_node(state: State, config: RunnableConfig):
    msg = "Curating Documents ...\n"
    prompt = collection_prompt("collection", state)
 
    relevant_urls = reasoning_llm.with_structured_output(ExtractInput).invoke(prompt)
    response = []
    for url_items in relevant_urls.urls:
        try:
            response.append(await tavily_client.extract(urls=url_items))
        
        except Exception as e:
            print(f"Error occurred during Tavily Extract request")
        
    return {"observations": response}

In [44]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

graph = StateGraph(State)

# Add nodes
graph.add_node("planner", planner_node)
graph.add_node("research_coordinator", research_coordinator_node)
graph.add_node("researcher",research_node)
graph.add_node("collector",collection_node)
graph.set_entry_point("planner")
graph.add_edge("research_coordinator", "researcher")
graph.add_edge("researcher", "collector")
graph.add_edge("collector", END)  # Option in the future, to add another step and filter the documents retrieved using rerhank before writing the report

agent = graph.compile(checkpointer=memory)

async for s in agent.astream(input=initial_state, config=config, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the rankings for the broader Seattle metro region's schools?  
Which stand out, which are lagging and what are some common themes or drivers for those that lead or lag?  
How should families evaluate their options including private schools in some of the key areas of the metro?
================================== Ai Message ==================================
Name: planner

{
    "thought": "The user wants a detailed research plan to understand school rankings in the broader Seattle metro region, identify standout and underperforming schools, analyze common factors influencing performance, and provide guidance for families evaluating school options, including private schools in key metro areas.",
    "title": "Research Plan for Seattle Metro Region School Rankings, Performance Drivers, and Family Evaluation Guidance",
    "steps": [
        {
            "title": "Collect Comprehensive Data on Scho

In [45]:
snapshot = agent.get_state(config)
{k for k, v in snapshot.values.items()}

{'current_plan', 'messages', 'observations', 'queries', 'resources'}

In [48]:
snapshot.values['observations'][:3]

[{'results': [{'url': 'https://www.niche.com/k12/search/best-public-high-schools/m/seattle-metro-area/',
    'raw_content': '2025 Best Public High Schools in the Seattle Area - Niche\n\n===============\n\n[Skip to Main Content](https://www.niche.com/k12/search/best-public-high-schools/m/seattle-metro-area/#maincontent)\n\n[](https://www.niche.com/?ref=k12 "Niche")\n\nK-12 Schools Seattle Area\n\nProvides auto-suggestions when entering text\n\nFind a school or district ...\n\nProvides auto-suggestions when entering text\n\nSearch in a district or a location ...\n\nLog In Sign Up\n\n*   [School Search](https://www.niche.com/k12/search/best-schools/m/seattle-metro-area/)\n*   [School Rankings](https://www.niche.com/k12/rankings/)\n*   [Schools Near You](https://www.niche.com/k12/schools-near-you/)\n\n[Write a review](https://www.niche.com/survey/hub/)\n\n[K-12 SCHOOLS](https://www.niche.com/k12/search/best-schools/m/seattle-metro-area/)\n\n*   [Review Your School](https://www.niche.com/k1

## Finished - Reporter

In [49]:

with open(f"reporter.md", "r", encoding="utf-8") as file:
    plan_prompt = file.read()
template = Template(plan_prompt)
question = snapshot.values['messages'][0].content
observations = snapshot.values['observations']
prompt = template.render(date=datetime.now().strftime('%d/%m/%Y'), question=question, observations=observations)

In [50]:
snapshot.values['messages'][0].content

"What are the rankings for the broader Seattle metro region's schools?  \nWhich stand out, which are lagging and what are some common themes or drivers for those that lead or lag?  \nHow should families evaluate their options including private schools in some of the key areas of the metro?"

In [51]:
def reporter_node(state: State, config: RunnableConfig):
    """Reporter node that write a final report."""
    print("Reporter write final report")
    
    with open(f"reporter.md", "r", encoding="utf-8") as file:
        plan_prompt = file.read()
    template = Template(plan_prompt)
    question = state.get('messages')[0].content
    observations = state.get('observations')
    prompt = template.render(date=datetime.now().strftime('%d/%m/%Y'), question=question, observations=observations)

    response = basic_llm.invoke(prompt)
    response_content = response.content
    print(f"reporter response: {response_content}")

    return {"final_report": response_content}

In [52]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

graph = StateGraph(State)

# Add nodes
graph.add_node("planner", planner_node)
graph.add_node("research_coordinator", research_coordinator_node)
graph.add_node("researcher",research_node)
graph.add_node("collector",collection_node)
graph.add_node("reporter",reporter_node)
graph.set_entry_point("planner")
graph.add_edge("research_coordinator", "researcher")
graph.add_edge("researcher", "collector")
graph.add_edge("collector", "reporter")
graph.add_edge("reporter", END)  # Option in the future, to add another step and filter the documents retrieved using rerhank before writing the report

agent = graph.compile(checkpointer=memory)

async for s in agent.astream(input=initial_state, config=config, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the rankings for the broader Seattle metro region's schools?  
Which stand out, which are lagging and what are some common themes or drivers for those that lead or lag?  
How should families evaluate their options including private schools in some of the key areas of the metro?
================================== Ai Message ==================================
Name: planner

{
    "thought": "The user wants a comprehensive analysis of school rankings in the broader Seattle metro region, identifying top-performing and lagging schools, exploring reasons behind their performance, and advising families on evaluating schooling options including private schools in key metro areas.",
    "title": "Research Plan for Analysis of Seattle Metro Region School Rankings and Evaluation",
    "steps": [
        {
            "title": "Collect Comprehensive School Performance Data and Rankings",
            "descrip

In [53]:
snapshot = agent.get_state(config)
{k for k, v in snapshot.values.items()}

{'current_plan',
 'final_report',
 'messages',
 'observations',
 'queries',
 'resources'}

In [54]:
display(Markdown(snapshot.values['final_report']))

**Executive Summary**

The Seattle metropolitan region (encompassing Seattle, Bellevue, Bothell, Redmond, and adjacent suburbs) presents a diverse educational landscape with a variety of public and private schooling options. The highest-ranked school districts are located predominantly on the Eastside (Bellevue, Issaquah, Mercer Island, Northshore, and Lake Washington districts), known for strong academic performance, comprehensive extracurricular programs, and high college readiness. Within Seattle city limits, the Seattle Public Schools district offers many option schools and specialized programs but faces challenges in educational equity and academic outcomes, motivating ongoing strategic plans focused on equity and inclusivity.

Ranking data from Niche.com (2025) positions these Eastside districts at the top with "A+" overall grades reflecting excellent academics, teacher quality, and graduation rates. Conversely, some districts such as Everett and Riverview lag behind with "B+" or "B" grades, often citing infrastructure or funding issues. Several themes emerge driving disparities, including socio-economic demographics, equity-focused initiatives, teacher quality, diversity, and family engagement. Families assessing school options should balance school rankings with programmatic fit, diversity, access to Spanish and dual-language immersion programs, and transportation logistics.

For private schools, the Seattle metro boasts prestigious and academically rigorous institutions, such as The Overlake School, Lakeside School, University Prep, Seattle Academy, and others mainly clustered around Seattle, Bellevue, and nearby areas. These schools emphasize college preparation, smaller student-to-teacher ratios, and specialized curricula (e.g., STEM, arts, religious education), though cost is an important consideration.

Seattle Public Schools offers an open enrollment and school choice system, allowing families to apply for attendance at option schools or outside their neighborhood zones subject to availability. These systems support access to diverse programs and help families tailor choices to needs, whether public, specialized option schools, or private alternatives. Evaluation should include academic outcomes, program availability, school climate, and logistical factors such as commute and family priorities.

---

### 1. Rankings in the Broader Seattle Metro Region

**Top-Ranked Public Districts**  
According to Niche’s 2025 rankings, the top public school districts in the Seattle metropolitan area include:^1

- **Bellevue School District** (Grade A+, 32 schools, ~19,000 students): Renowned for academic rigor, supportive staff, well-funded extracurriculars, and athlete prioritization. It holds highly rated high schools such as Bellevue High and several specialized programs.  
- **Northshore School District** (Grade A+, 39 schools, ~22,900 students): High marks for diversity, strong remote learning adaptation, but some concerns over specific arts funding.  
- **Lake Washington School District** (Grade A+, 57 schools, ~31,000 students): Praised for supportive teachers and counselors, safety, and diverse student population. Home to Tesla STEM High School, ranked #1 high school in the region by US News.  
- **Issaquah School District** (Grade A+, 30 schools, ~19,500 students): Academically focused with an emphasis on college preparation and social responsibility.  
- **Mercer Island School District** (Grade A, 6 schools, ~4,000 students): Smaller district with personalized attention, high academic and social-emotional support, and excellent college placements.  
- **Bainbridge Island School District** (Grade A, 10 schools, ~3,600 students): Known for flexible and responsive teachers and competitive sports.

Other districts with solid "A" or "A-" grades include Shoreline, Tahoma, Vashon Island, University Place, Snoqualmie Valley, Edmonds, and Everett (the latter with slightly lower rankings and challenges).

**Lower-Ranked Districts and Challenges**  
Districts like **Everett, Riverview, Mukilteo, Sumner-Bonney Lake, and Fife** rank lower (B+ or B), with some citing funding constraints, aging facilities, and uneven program offerings. The Seattle Public Schools district, while large (109 schools, 51,000+ students), carries an overall Niche grade of A- and must address significant equity challenges and achievement gaps, particularly among students of color, as highlighted in district strategic plans.^2

---

### 2. Standout Schools and Those Lagging

**Standouts**  
- **Tesla STEM High School (Lake Washington SD):** #1 High School in Seattle area and ranked #3 nationally by U.S. News.^3 A specialized STEM program with 100% college readiness, strong AP participation, and modern facilities.  
- **International School (Bellevue SD):** Integrates global curriculum with rigorous academics, ranked #2 in Seattle metro area by U.S. News.  
- **Private Schools like The Overlake School, Lakeside School, Eastside Preparatory, University Prep:** All ranked "A+" by Niche with low student-teacher ratios (5-13:1) and strong college preparatory focus.^4

**Lagging Schools/Districts**  
- **Everett SD:** Despite a large district size and many programs, struggles with funding cuts and disparate educational outcomes.  
- **Seattle Public Schools (SPS):** Largest district facing socio-economic disparities and achievement gaps. The district acknowledges systemic inequities affecting African American boys and students of color.  
- **Some smaller districts and more rural areas:** May have fewer resources or specialized programs.

---

### 3. Common Themes and Drivers for Rankings

**Equity and Inclusion Focus**  
Seattle’s district strategic plans (notably Seattle Excellence 2019-2024, ongoing through 2025) emphasize reducing opportunity and achievement gaps for Students of Color, especially African American boys and teens.^2 It includes culturally responsive teaching, restorative discipline practices, and expanded family engagement.

**Academic Rigor and Curriculum**  
High-ranking districts and schools consistently offer rigorous core academics, Advanced Placement (AP) and International Baccalaureate (IB) opportunities, gifted and talented programs, and specialty schools (e.g., magnet STEM campuses).

**Teacher Quality and Support**  
Positive reviews consistently highlight caring, supportive teachers and administrators who engage with students and families, contributing directly to student success.

**Diversity and Community Fit**  
Diverse student populations, inclusive environments, and alignment with family values (religious, socio-economic, cultural) play a central role in school desirability and success.

**Extracurricular and Enrichment Opportunities**  
Strong programs in arts, sports, leadership, and specialized areas such as aviation, computer science, or performing arts support holistic student development.

**Funding and Resources**  
Districts with greater financial resources typically sustain superior facilities, smaller class sizes, and richer program offerings. Conversely, funding shortfalls correlate with lagging performance.

---

### 4. How Families Should Evaluate Their Options

**Understand School and Neighborhood Fit**  
Families should look beyond raw rankings, considering school culture, size, diversity, and availability of programs (e.g., dual language immersion, specialty STEM or arts). Local real estate and commute considerations matter greatly in daily life and child wellbeing.

**Public vs. Private Considerations**  
- **Public Schools:** Offer broad access to diverse populations, often with strong neighborhood or choice-based programs. Seattle Public Schools provide open enrollment and option schools for specialized learning environments.  
- **Private Schools:** Tend to feature smaller class sizes, college preparatory curricula, often religious or philosophical orientations, and require significant tuition. They may suit families prioritizing academic prestige, specific cultural/religious identities, or personalized attention.

**Use Multiple Data Sources and Visits**  
Resources such as Niche.com and GreatSchools.org provide valuable data on academics, teacher quality, safety, and reviews. Visiting schools, talking with administrators and parents, and assessing student needs are essential steps. Insights from local families underline the importance of “fit” and family values in decision-making.^5

**Transportation and Logistics**  
Consider commute times, availability of school buses, proximity to home or work, and the Seattle area’s traffic patterns. Some private schools provide shuttle services, while public school assignments may depend on neighborhood boundaries or lottery options.

**Special Programs and Supports**  
Families with students requiring special education, ESL, or gifted services should evaluate districts and schools offering comprehensive supports. For students with disabilities, Seattle offers service schools and partner programs. Homeschooling remains a flexible option with district co-ops for social and academic enrichment.

---

### 5. Rankings of Key Private Schools in Seattle Metro

According to Niche’s 2025 rankings of private high schools:^4

- **The Overlake School (Redmond):** #1 private high school in Seattle metro, A+ grade, 7:1 student-teacher ratio, strong college prep.  
- **Eastside Preparatory School (Kirkland):** #2, highly rated for teaching and activities.  
- **Lakeside School (Seattle):** #3, prestigious, rigorous academics, historic clientele.  
- **Annie Wright Schools (Tacoma):** #4, strong community, academic and travel opportunities.  
- **University Prep (Seattle):** #5, excellent academics and inclusive environment.  
- **The Bear Creek School (Redmond):** #6, academically rigorous with caring community.  
- **Seattle Academy and The Northwest School (Seattle):** Among top ten, known for strong arts and college prep.  
- **Holy Names Academy and Bishop Blanchet (Seattle):** Prominent faith-based options with robust academics.

Private schools generally exhibit low student-teacher ratios (often < 10:1), emphasis on AP/IB programs, and highly engaged teaching staff. Tuition and cost considerations are relevant, with many schools offering financial aid to widen access.

---

### Conclusion

The Seattle metropolitan region offers a rich array of educational choices for families, with strong public districts concentrated east of Lake Washington and a varied set of option schools and magnet programs within Seattle proper. Top-performing districts like Bellevue, Northshore, Issaquah, and Lake Washington set the bar in academics and extracurricular breadth, buoyed by resources and community engagement.

Families should evaluate school options based on academic quality, fit with social-emotional needs and values, diversity, available supports, and practical concerns such as commute and enrollment policies. The city’s open enrollment and school choice frameworks provide flexibility within public education, while a diverse private school landscape offers alternatives emphasizing college prep and specialized pedagogies.

Seattle Public Schools has recognized challenges related to equity and inclusion, especially for students of color, and is actively pursuing strategic improvement plans focused on systemic change and culturally responsive education. This creates an improving but still complex public school environment—making personal research and engagement critical for families navigating options.

In sum, the best approach for families is to utilize data sources such as Niche and US News rankings, partake in school visits and community feedback, understand the various program offerings and enrollment procedures, and weigh public and private options carefully against their children’s unique needs and family circumstances.

---

### References

1. Niche. (2025). *Best School Districts in Seattle Area*. Retrieved from [Niche.com](https://www.niche.com/k12/search/best-school-districts/m/seattle-metro-area/)  
2. Seattle Public Schools. (2025). *Strategic Plan and Equity Initiatives*. Retrieved from [seattleschools.org/about/strategic-plan](https://www.seattleschools.org/about/strategic-plan/)  
3. U.S. News & World Report. (2025). *Best High Schools in Seattle, WA Area*. Retrieved from [usnews.com/education/best-high-schools/washington/rankings/seattle-wa-42660](https://www.usnews.com/education/best-high-schools/washington/rankings/seattle-wa-42660)  
4. Niche. (2025). *Best Private High Schools in Seattle Area*. Retrieved from [niche.com/k12/search/best-private-high-schools/m/seattle-metro-area/](https://www.niche.com/k12/search/best-private-high-schools/m/seattle-metro-area/)  
5. ParentMap. (2024). *Should We Choose Public or Private School?* Retrieved from [parentmap.com/article/seattle-public-private-school-options](https://www.parentmap.com/article/seattle-public-private-school-options)  

---

Please let me know if you would like a deeper dive into specific school profiles, program options, or enrollment procedures in the Seattle metro region.